## This Phi-3 Fine Tuning Notebook provides instructions on how to:

- Fine-tune the Phi-3 mini model using QLoRA and LoRA techniques
- Quantize the Phi-3 mini model using BitsandBytes and GPTQ for efficient memory usage
- Execute the Phi-3 mini model using Hugging Face's Transformers library
- Each section in this notebook is designed to be run independently, allowing you to focus on specific tasks as needed.



## Fine-Tuning Phi-3
Welcome to the guide on fine-tuning the Phi-3 model. Phi-3 is a powerful language model developed by Microsoft, designed to generate human-like text based on the input it receives. Fine-tuning is a process that involves training a pre-trained model (like Phi-3) on a specific task, allowing it to adapt its pre-learned knowledge to the new task.

In this guide, we will walk you through the steps of fine-tuning the Phi-3 model. This process can help improve the model's performance on specific tasks or domains that were not covered in its original training data.

The fine-tuning process involves several steps, including setting up the environment, loading the pre-trained model, preparing the training data, and finally, training the model on the new data.

By the end of this guide, you should have a good understanding of how to fine-tune the Phi-3 model for your specific needs. Let's get started!

# Inference

This section demonstrates how to run inference using the Phi-3 mini model with Hugging Face's Transformers library, specifically the 16-bit version.

In [ ]:
# This command is run in a bash shell due to '%%bash' at the beginning.
# 'pip install -qqq' is used to install Python packages with pip, Python's package installer, in a less verbose mode.
# 'accelerate', 'transformers', 'auto-gptq', and 'optimum' are the packages being installed.
# These packages are necessary for the fine-tuning and inference of the Phi-3 model.
%%bash
pip install -qqq accelerate transformers auto-gptq optimum

Using the original model (16-bit version)

It requires 7.4 GB of GPU RAM

In [ ]:
# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

# Set a seed for reproducibility
set_seed(2024)

# Define the prompt for the model
prompt = "insert your prompt here"

# Define the model checkpoint simply replace with Phi-3 Model Required
model_checkpoint = "microsoft/Phi-3-mini-4k-instruct"

# Load the tokenizer from the model checkpoint
# trust_remote_code=True allows the execution of code from the model files
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,trust_remote_code=True)

# Load the model from the model checkpoint
# trust_remote_code=True allows the execution of code from the model files
# torch_dtype="auto" automatically determines the appropriate torch.dtype
# device_map="cuda" specifies that the model should be loaded to the GPU
model = AutoModelForCausalLM.from_pretrained(model_checkpoint,
                                             trust_remote_code=True,
                                             torch_dtype="auto",
                                             device_map="cuda")

# Tokenize the prompt and move the tensors to the GPU
inputs = tokenizer(prompt,
                   return_tensors="pt").to("cuda")

# Generate a response from the model
# do_sample=True means the model will generate text by sampling from the distribution of possible outputs
# max_new_tokens=120 limits the length of the generated text to 120 tokens
outputs = model.generate(**inputs,
                         do_sample=True, max_new_tokens=120)

# Decode the generated tokens and remove any special tokens
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Print the generated response from the model
print(response)

## Code Generation

In [ ]:
# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

# Set a seed for reproducibility
set_seed(2024)

# Define the prompt for the model. In this case, the prompt is a request for C# code.
prompt = "Write a C# code that reads the content of multiple text files and save the result as CSV"

# Define the model checkpoint and Phi-3 Model Required
model_checkpoint = "microsoft/Phi-3-mini-4k-instruct"

# Load the tokenizer from the model checkpoint
# trust_remote_code=True allows the execution of code from the model files
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,trust_remote_code=True)

# Load the model from the model checkpoint
# trust_remote_code=True allows the execution of code from the model files
# torch_dtype="auto" automatically determines the appropriate torch.dtype
# device_map="cuda" specifies that the model should be loaded to the GPU
model = AutoModelForCausalLM.from_pretrained(model_checkpoint,
                                             trust_remote_code=True,
                                             torch_dtype="auto",
                                             device_map="cuda")

# Tokenize the prompt and move the tensors to the GPU
inputs = tokenizer(prompt,
                   return_tensors="pt").to("cuda")

# Generate a response from the model
# do_sample=True means the model will generate text by sampling from the distribution of possible outputs
# max_new_tokens=200 limits the length of the generated text to 200 tokens
outputs = model.generate(**inputs,
                         do_sample=True, max_new_tokens=200)

# Decode the generated tokens and remove any special tokens
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# Print the generated response from the model
print(response)

# Quantization

When the Phi-3 model is fine-tuned using Hugging Face's Transformers and subsequently quantized using 4-bit GPTQ, it necessitates 2.7 GB of GPU RAM.4

"Bitsandbytes NF4" is a specific configuration or method within the Bitsandbytes library, which is used for quantization. Quantization is a process that reduces the numerical precision of the weights in a model to make it smaller and faster. 

In [ ]:
Bitsandbytes NF4

In [ ]:
# This command is used to install and upgrade necessary Python packages using pip, Python's package installer.
# The '!' at the beginning allows you to run shell commands in the notebook.
# '-qqq' is used to make the installation process less verbose.
# '--upgrade' ensures that if the packages are already installed, they are upgraded to the latest version.
# 'transformers', 'bitsandbytes', 'accelerate', and 'datasets' are the packages being installed/upgraded.
!pip install -qqq --upgrade transformers bitsandbytes accelerate datasets

# Phi-3 Fine-tuning

In [ ]:
# This command is run in a bash shell due to '%%bash' at the beginning.
# 'pip -q install' is used to install Python packages with pip, Python's package installer, in a quiet mode which reduces the output verbosity.
# 'huggingface_hub', 'transformers', 'peft', and 'bitsandbytes' are the packages being installed by the first command.
# These packages are necessary for the fine-tuning and inference of the Phi-3 model.
# 'trl' and 'xformers' are additional packages being installed by the second command.
# 'datasets' is a package for providing access to a vast range of datasets, installed by the third command.
# The last command ensures that 'torch' version is at least 1.10. If it's already installed but the version is lower, it will be upgraded.
%%bash
pip -q install huggingface_hub transformers peft bitsandbytes
pip -q install trl xformers
pip -q install datasets
pip install torch>=1.10

In [ ]:
# Import necessary modules from the transformers library
# AutoModelForCausalLM: This is a class for causal language models. It's used for tasks like text generation.
# AutoTokenizer: This class is used for tokenizing input data, a necessary step before feeding data into a model.
# TrainingArguments: This class is used for defining the parameters for model training, like learning rate, batch size, etc.
# BitsAndBytesConfig: This class is used for configuring the BitsAndBytes quantization process.
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig

# Import necessary modules from the huggingface_hub library
# ModelCard: This class is used for creating a model card, which provides information about a model.
# ModelCardData: This class is used for defining the data of a model card.
# HfApi: This class provides an interface to the Hugging Face API, allowing you to interact with the Hugging Face Model Hub.
from huggingface_hub import ModelCard, ModelCardData, HfApi

# Import the load_dataset function from the datasets library. This function is used for loading datasets.
from datasets import load_dataset

# Import the Template class from the jinja2 library. This class is used for creating dynamic HTML templates.
from jinja2 import Template

# Import the SFTTrainer class from the trl library. This class is used for training models.
from trl import SFTTrainer

# Import the yaml module. This module is used for working with YAML files.
import yaml

# Import the torch library. This library provides tools for training and running deep learning models.
import torch

In [ ]:
# MODEL_ID is a string that specifies the identifier of the pre-trained model that will be fine-tuned. 
# In this case, the model is 'Phi-3-mini-4k-instruct' from Microsoft.
MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"

# NEW_MODEL_NAME is a string that specifies the name of the new model after fine-tuning.
# Here, the new model will be named 'opus-samantha-phi-3-mini-4k'.
NEW_MODEL_NAME = "New-Model-phi-3-mini-4k"

In [ ]:
# DATASET_NAME is a string that specifies the name of the dataset to be used for fine-tuning.
# Replace "replace with your dataset" with the actual name of your dataset.
DATASET_NAME = "replace with your dataset"

# SPLIT specifies the portion of the dataset to be used. In this case, the 'train' split of the dataset will be used.
SPLIT = "train"

# MAX_SEQ_LENGTH is an integer that specifies the maximum length of the sequences that the model will handle.
MAX_SEQ_LENGTH = 2048

# num_train_epochs is an integer that specifies the number of times the training process will go through the entire dataset.
num_train_epochs = 1

# license is a string that specifies the license under which the model is distributed. In this case, it's Apache License 2.0.
license = "apache-2.0"

# username is a string that specifies the GitHub username of the person who is fine-tuning the model.
username = "GitHubUsername"

# learning_rate is a float that specifies the learning rate to be used during training.
learning_rate = 1.41e-5

# per_device_train_batch_size is an integer that specifies the number of samples to work through before updating the internal model parameters.
per_device_train_batch_size = 4

# gradient_accumulation_steps is an integer that specifies the number of steps to accumulate gradients before performing a backward/update pass.
gradient_accumulation_steps = 1

In [ ]:
# This code checks if the current CUDA device supports bfloat16 (Brain Floating Point) computations.
# If bfloat16 is supported, it sets the compute_dtype to torch.bfloat16.
# If not, it sets the compute_dtype to torch.float16.
# bfloat16 and float16 are both half-precision floating-point formats, but bfloat16 provides better performance on some hardware.
if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
else:
  compute_dtype = torch.float16

In [ ]:
# Load the pre-trained model specified by MODEL_ID using the AutoModelForCausalLM class.
# The 'trust_remote_code=True' argument allows the execution of code from the model card (if any).
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, trust_remote_code=True)

# Load the tokenizer associated with the pre-trained model specified by MODEL_ID using the AutoTokenizer class.
# The 'trust_remote_code=True' argument allows the execution of code from the model card (if any).
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

# Load the dataset specified by DATASET_NAME using the load_dataset function.
# The 'split="train"' argument specifies that we want to load the training split of the dataset.
dataset = load_dataset(DATASET_NAME, split="train")

# Get the ID of the end-of-sentence (EOS) token from the tokenizer and store it in EOS_TOKEN.
# This token is used to mark the end of a sentence in the input data.
EOS_TOKEN=tokenizer.eos_token_id

In [ ]:
# This line simply prints the contents of the 'dataset' variable.
# 'dataset' is expected to be a Dataset object loaded from the 'datasets' library.
# Printing it will display information about the dataset such as the number of samples, the features, and a few example data points.
dataset

In [ ]:
# Select a subset of the data for faster processing
dataset = dataset.select(range(100))

In [ ]:
# This line simply prints the contents of the 'dataset' variable.
# 'dataset' is expected to be a Dataset object loaded from the 'datasets' library.
# Printing it will display information about the dataset such as the number of samples, the features, and a few example data points.
dataset

In [ ]:
# Define a function to format the prompts in the dataset.
# This function takes a batch of examples and returns a dictionary with the key 'text' and the value being a list of formatted texts.
def formatting_prompts_func(examples):
    # Extract the conversations from the examples.
    convos = examples["conversations"]
    # Initialize an empty list to store the formatted texts.
    texts = []
    # Define a dictionary to map the 'from' field in the conversation to a prefix.
    mapper = {"system": "system\n", "human": "\nuser\n", "gpt": "\nassistant\n"}
    # Define a dictionary to map the 'from' field in the conversation to a suffix.
    end_mapper = {"system": "", "human": "", "gpt": ""}
    # Iterate over each conversation.
    for convo in convos:
        # Format the conversation by joining each turn with its corresponding prefix and suffix.
        # Append the EOS token to the end of the conversation.
        text = "".join(f"{mapper[(turn := x['from'])]} {x['value']}\n{end_mapper[turn]}" for x in convo)
        texts.append(f"{text}{EOS_TOKEN}")
    # Return the formatted texts.
    return {"text": texts}

# Apply the formatting function to the dataset using the map method.
# The 'batched=True' argument means that the function is applied to batches of examples.
dataset = dataset.map(formatting_prompts_func, batched=True)

# Print the 9th example from the 'text' field of the dataset to check the result.
print(dataset['text'][8])

In [ ]:
# Create a TrainingArguments object, which is used to define the parameters for model training.

args = TrainingArguments(
    # 'evaluation_strategy' is set to "steps", which means evaluation is done at each logging step.
    evaluation_strategy="steps",

    # 'per_device_train_batch_size' is set to 7, which means each training batch will contain 7 samples per device.
    per_device_train_batch_size=7,

    # 'gradient_accumulation_steps' is set to 4, which means gradients are accumulated for 4 steps before performing a backward/update pass.
    gradient_accumulation_steps=4,

    # 'gradient_checkpointing' is set to True, which means model gradients are stored in memory during training to reduce memory usage.
    gradient_checkpointing=True,

    # 'learning_rate' is set to 1e-4, which is the learning rate for the optimizer.
    learning_rate=1e-4,

    # 'fp16' is set to True if bfloat16 is not supported, which means the model will use 16-bit floating point precision for training if possible.
    fp16 = not torch.cuda.is_bf16_supported(),

    # 'bf16' is set to True if bfloat16 is supported, which means the model will use bfloat16 precision for training if possible.
    bf16 = torch.cuda.is_bf16_supported(),

    # 'max_steps' is set to -1, which means there is no maximum number of training steps.
    max_steps=-1,

    # 'num_train_epochs' is set to 3, which means the training process will go through the entire dataset 3 times.
    num_train_epochs=3,

    # 'save_strategy' is set to "epoch", which means the model is saved at the end of each epoch.
    save_strategy="epoch",

    # 'logging_steps' is set to 10, which means logging is done every 10 steps.
    logging_steps=10,

    # 'output_dir' is set to NEW_MODEL_NAME, which is the directory where the model and its configuration will be saved.
    output_dir=NEW_MODEL_NAME,

    # 'optim' is set to "paged_adamw_32bit", which is the optimizer to be used for training.
    optim="paged_adamw_32bit",

    # 'lr_scheduler_type' is set to "linear", which means the learning rate scheduler type is linear.
    lr_scheduler_type="linear"
)

In [ ]:
# Create an instance of the SFTTrainer class, which is used to fine-tune the model.

trainer = SFTTrainer(
    # 'model' is the pre-trained model that will be fine-tuned.
    model=model,

    # 'args' are the training arguments that specify the training parameters.
    args=args,

    # 'train_dataset' is the dataset that will be used for training.
    train_dataset=dataset,

    # 'dataset_text_field' is the key in the dataset that contains the text data.
    dataset_text_field="text",

    # 'max_seq_length' is the maximum length of the sequences that the model will handle.
    max_seq_length=128,

    # 'formatting_func' is the function that will be used to format the prompts in the dataset.
    formatting_func=formatting_prompts_func
)

In [ ]:
# 'device' is set to 'cuda', which means the CUDA device will be used for computations if available.
device = 'cuda'

# Import the 'gc' module, which provides an interface to the garbage collector.
import gc

# Import the 'os' module, which provides a way of using operating system dependent functionality.
import os

# Call the 'collect' method of the 'gc' module to start a garbage collection, which can help free up memory.
gc.collect()

# Call the 'empty_cache' method of 'torch.cuda' to release all unused cached memory from PyTorch so that it can be used by other GPU applications.
torch.cuda.empty_cache()

In [ ]:
# Call the 'train' method of the 'trainer' object to start the training process.
# This method will fine-tune the model on the training dataset according to the parameters specified in the 'args' object.
trainer.train()